In [1]:
import os
import cv2
import pickle
import numpy as np
from datetime import datetime
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


data = []
labels = []
dataset_path = r"E:\FER\images\train"  # <- match your folder name exactly
emotion_labels = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]


# Load and preprocess images
for emotion in emotion_labels:
    folder = os.path.join(dataset_path, emotion)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, (48, 48))
        data.append(img)
        labels.append(emotion)

data = np.array(data).reshape(-1, 48, 48, 1) / 255.0

# Encode labels and determine number of classes
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)  # get number of unique classes
labels = to_categorical(labels_encoded)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels)

# Build CNN model
model = Sequential([
    Input(shape=(48, 48, 1)),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

os.makedirs("model", exist_ok=True)  # ensures the 'model' folder exists
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), shuffle=True)
model.save("model/emotion_model.h5")

with open("model/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)
    
print("Model trained and saved.")


Epoch 1/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 242s 555ms/step - accuracy: 0.1694 - loss: 2.8410 - val_accuracy: 0.1904 - val_loss: 2.2017
Epoch 2/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 238s 556ms/step - accuracy: 0.2091 - loss: 2.1288 - val_accuracy: 0.2255 - val_loss: 2.0390
Epoch 3/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 237s 554ms/step - accuracy: 0.2227 - loss: 2.0745 - val_accuracy: 0.2436 - val_loss: 2.0043
Epoch 4/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 243s 568ms/step - accuracy: 0.2221 - loss: 2.0324 - val_accuracy: 0.3170 - val_loss: 1.9399
Epoch 5/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 243s 568ms/step - accuracy: 0.2470 - loss: 1.9907 - val_accuracy: 0.3084 - val_loss: 1.9189
Epoch 6/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 241s 564ms/step - accuracy: 0.2548 - loss: 1.9529 - val_accuracy: 0.2851 - val_loss: 1.9018
Epoch 7/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 241s 563ms/step - accuracy: 0.2546 - loss: 1.9325 - val_accuracy: 0.3057 - val_loss: 1.9284
Epoch 8/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 243s 568ms/step - accuracy: 0.2731 -

Model trained and saved.


In [ ]:
import pandas as pd

print("Starting Real-Time Emotion Detection")

with open("model/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)
emotion_labels = label_encoder.classes_

model = load_model("model/emotion_model.h5")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

cap = cv2.VideoCapture(1)
cap.set(3, 640)  # width
cap.set(4, 480)  # height

log_data = []
excel_file = "emotion_log.xlsx"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face.astype("float") / 255.0
        face = img_to_array(face)
        face = np.expand_dims(face, axis=0)

        preds = model.predict(face, verbose=0)[0]
        label = emotion_labels[np.argmax(preds)]
        confidence = np.max(preds)
        
        cv2.putText(frame, f"{label} ({int(confidence*100)}%)", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        #logs the emotion timing along with the confidence
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_data.append({
            'Time': timestamp,
            'Emotion': label,
            'Confidence': round(confidence * 100, 2)
        })

    cv2.imshow("Real-time Emotion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

if log_data:
    df = pd.DataFrame(log_data)
    df.to_excel(excel_file, index=False)
    print(f"Emotion logs saved to {excel_file}")
else:
    print("No emotions detected to log.")

Starting Real-Time Emotion Detection


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

test_data_path = r"E:\FER\images\validation"  
model_path = "model/emotion_model.h5"
encoder_path = "model/label_encoder.pkl"

# Load label encoder and model
with open(encoder_path, "rb") as f:
    label_encoder = pickle.load(f)
model = load_model(model_path)

# Load test images
X_test = []
y_true_labels = []

emotion_labels = label_encoder.classes_

for emotion in emotion_labels:
    folder = os.path.join(test_data_path, emotion)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, (48, 48))
        X_test.append(img)
        y_true_labels.append(emotion)

X_test = np.array(X_test).reshape(-1, 48, 48, 1) / 255.0
y_true_encoded = label_encoder.transform(y_true_labels)
y_true_categorical = to_categorical(y_true_encoded)

# Predictions
y_pred_probs = model.predict(X_test)
y_pred_encoded = np.argmax(y_pred_probs, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_encoded)

# Classification Report
print("\nClassification Report:\n")
print(classification_report(y_true_labels, y_pred_labels))

# Confusion Matrix
cm = confusion_matrix(y_true_labels, y_pred_labels, labels=emotion_labels)

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=emotion_labels, yticklabels=emotion_labels, cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

# Plot Classification Report as bar chart
report = classification_report(y_true_labels, y_pred_labels, output_dict=True)
metrics = ['precision', 'recall', 'f1-score']

for metric in metrics:
    values = [report[label][metric] for label in emotion_labels]
    plt.figure(figsize=(10, 4))
    sns.barplot(x=emotion_labels, y=values)
    plt.title(f"{metric.title()} per Emotion")
    plt.ylabel(metric.title())
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()